In [5]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)

# Embedding mode
there are lots of embedding models, you can see which one is performing good on HF leaderboard and suits your needs
https://huggingface.co/spaces/mteb/leaderboard


the max token is important for our chunking word limit, token are about 0.75 word

In [6]:
# Read data
df = pd.read_csv("./data/chunks_data.csv")
df["text"] = df["text"].astype(str)
df["title"] = df["title"].astype(str)
df

,id,text,title,ns
0,2030-000,"Portal/LOL Cleanup| Game modes section links to ], which is legacy terminology. Game modes section includes TFT but describes TFT as a ""game available through the League Client"" and the ] page des...",League of Legends,0
1,2030-001,main|Summoner's Rift (League of Legends) '''Summoner's Rift''' resembles the ''Defense of the Ancients'' map with three lanes and supports five players per side. The maps put two teams with a fixe...,League of Legends,0
2,2030-002,These items boost stats of the champion. Some items can be combined into more powerful items by following predefined recipes. ;Original Champion Roster Column|3| {{ci|Alistar ci|Amumu ci|Anivia ci...,League of Legends,0
3,2030-003,- IT'S ON Gameplay Trailer| Champions in Season 2021- Dev Video - League of Legends| Esports in Season 2021- Esports - Riot Games| Gameplay in Season 2021 - Dev Video - League of Legends| Skins & ...,League of Legends,0
4,2032-000,":''This is about the tip|League of Legends player-controlled units, and their depiction in the ]. For other uses of '''Champion''', see ]'' ).]] '''Champions''' are the player-controlled character...",Champion,0
...,...,...,...,...
16219,1631196-000,"PoC item infobox|custom|nameSuccubus's Brand|image:Succubus's Brand LoR relic.png|imagesize200px|type:Relic|desc+1|+1 and when I kill a unit summon a random ].|rarity:Rare Change Log :{| class""art...",Succubus's Brand (The Path of Champions),0
16220,1631197-000,"PoC item infobox|custom|nameVoidborne Carapace|image:Voidborne Carapace LoR relic.png|imagesize200px|type:Relic|desc{{tipLoR|EvolveWhen ANY unit dies, grant me its keywords.|rarity:Rare}} Change L...",Voidborne Carapace (The Path of Champions),0
16221,1631198-000,"PoC item infobox|custom|nameArcane Knowledge|image:06RU043-full.png|imagesize200px|type:Item|descWhen I'm summoned, draw a spell.|rarity:Common|Unitstrue: Change Log {| class:""article-table ruling...",Arcane Knowledge (The Path of Champions),0
16222,1631199-000,"PoC item infobox|custom|nameImmortal|image:08NX028-full.png|imagesize200px|type:Item|desc+1|+0 and {{tipLoR|Deathless.|rarity:Common|Unitstrue}}: Change Log {| class:""article-table ruling-table""! ...",Immortal (The Path of Champions),0


In [7]:
input = "This chunk is about " + df['title']  + f" with ID-" + df['id'].astype(str) + ": "  + df["text"]

In [8]:
input = input.astype(str)
input

0        This chunk is about League of Legends with ID-2030-000: Portal/LOL Cleanup| Game modes section links to ], which is legacy terminology. Game modes section includes TFT but describes TFT as a "game...
1        This chunk is about League of Legends with ID-2030-001: main|Summoner's Rift (League of Legends) '''Summoner's Rift''' resembles the ''Defense of the Ancients'' map with three lanes and supports f...
2        This chunk is about League of Legends with ID-2030-002: These items boost stats of the champion. Some items can be combined into more powerful items by following predefined recipes. ;Original Cham...
3        This chunk is about League of Legends with ID-2030-003: - IT'S ON Gameplay Trailer| Champions in Season 2021- Dev Video - League of Legends| Esports in Season 2021- Esports - Riot Games| Gameplay ...
4        This chunk is about Champion with ID-2032-000: :''This is about the tip|League of Legends player-controlled units, and their depiction in the ]. For other 

In [9]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True, model_kwargs={"torch_dtype":torch.bfloat16})

embeddings = model.encode(input, batch_size=1, show_progress_bar=True)
embeddings_torch = torch.from_numpy(embeddings).to(device)

/home/userzero/.local/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-22 14:57:32.891272: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 14:57:32.921716: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 14:57:32.921743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been

Batches:   0%|          | 0/16224 [00:00<?, ?it/s]

In [10]:
embeddings.shape

(16224, 768)

In [13]:
text = "Kayn weapon name?"

text_embeddings = model.encode(text, batch_size=1, show_progress_bar=True)
text_embeddings_torch = torch.from_numpy(text_embeddings).to(device)

cos_sim_score = cos_sim(a=text_embeddings_torch, b=embeddings_torch)[0]

top_result = torch.topk(cos_sim_score, k=5)
top_result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.return_types.topk(
values=tensor([0.6318, 0.6301, 0.6259, 0.6227, 0.6189], device='cuda:0'),
indices=tensor([5476, 6342, 5488, 3198, 5474], device='cuda:0'))

In [14]:
import textwrap

for i in top_result[1]:
    print("title: " + df.iloc[int(i)]["title"] + ": " )
    print(textwrap.fill(df.iloc[int(i)]["text"], width=70))
    print("___________________________\n\n")

title: Kayn/LoL/Audio: 
mercy! Right, Noxian?"'' ;Scoring First Blood
sm2|Kayn_Original_KillFirst_0.ogg|Kayn ''"Merely the first to die.
There will be more."'' sm2|Kayn_Original_KillFirst_1.ogg|Kayn ''"That
was for Zed."'' sm2|Kayn_Original_KillFirst_2.ogg|Rhaast ''"You never
forget your first kill... Nope, that's not right. I've already
forgotten it."'' sm2|Kayn_Original_KillFirst_3.ogg|Rhaast ''"Ooohhh!
That felt good."'' ;Scoring a Quadra kill
sm2|Kayn_Original_KillQuadra_1.ogg|Kayn ''"That's four by my count."''
sm2|Kayn_Original_KillQuadra_0.ogg|Rhaast ''"Stand back! Kayn doesn't
know how deadly this thing gets!"'' ;Scoring a Pentakill
sm2|Kayn_Original_KillPenta_0.ogg|Kayn ''"You wanted mercy? Not from
me!"'' sm2|Kayn_Original_KillPenta_1.ogg|Rhaast ''"Where are the
worthy?"'' ccib|Gold.png|size:32Shopping ;Buying an Item
sm2|Kayn_Original_BuyItem_4.ogg|Kayn ''"This will very much upset the
balance."'' sm2|Kayn_Original_BuyItem_6.ogg|Kayn ''"A match made in
shadow."'' sm2|Kayn_Or

In [15]:
torch.save(embeddings_torch, "./data/embeddings_torch.pt")

: 